In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [43]:
# Importing the original understat dataframes and sorting them by date

eng_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')
eng_df = eng_df.sort_values(by='date',ascending=False)
fra_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='gamesheet')
fra_df = fra_df.sort_values(by='date',ascending=False)
ger_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='gamesheet')
ger_df = ger_df.sort_values(by='date',ascending=False)
ita_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='gamesheet')
ita_df = ita_df.sort_values(by='date',ascending=False)
spa_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='gamesheet')
spa_df = spa_df.sort_values(by='date',ascending=False)

In [44]:
# Form DF (Last 10 games)

def last10(df):
    temp_df = df
    temp_df['played'] = 1
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['date_int'] = (datetime.datetime.now()) - (temp_df['date'])
    temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x.nsmallest(10, 'date_int'))
    temp_df.reset_index(inplace=True)
    temp_df.drop(['level_0','level_1','date_int'],axis=1,inplace=True)
    return temp_df

eng_l10 = last10(eng_df)
fra_l10 = last10(fra_df)
ger_l10 = last10(ger_df)
ita_l10 = last10(ita_df)
spa_l10 = last10(spa_df)

C:\Users\User\AppData\Local\Temp\ipykernel_7548\3480250055.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x.nsmallest(10, 'date_int'))
C:\Users\User\AppData\Local\Temp\ipykernel_7548\3480250055.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = temp_df.groupby(['Team'],as_index=False).apply(lambda x: x.nsmallest(10, 'date_int

In [45]:
# Home/Away Tables last 10 games (Form)

def home_away_table_form(df):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1
    
    # new df
    df_h_a = df.groupby(["Team","H/A"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby("H/A")]
    away_table = h_a_sep[0].droplevel("H/A")
    home_table = h_a_sep[1].droplevel("H/A")
    home_table = home_table.apply(pd.to_numeric)
    away_table = away_table.apply(pd.to_numeric)
    home_table['GD'] = home_table['G'] - home_table['GC']
    away_table['GD'] = away_table['G'] - away_table['GC']
    home_table['PPG'] = round(home_table['Pts'] / home_table['played'],2)
    away_table['PPG'] = round(away_table['Pts'] / away_table['played'],2)
    home_table['BTTS%'] = round(home_table['BTTS'] / home_table['played'],2)
    away_table['BTTS%'] = round(away_table['BTTS'] / away_table['played'],2)
    home_table['Blank%'] = round(home_table['Blank'] / home_table['played'],2)
    away_table['Blank%'] = round(away_table['Blank'] / away_table['played'],2)
    home_table['CS%'] = round(home_table['CS'] / home_table['played'],2)
    away_table['CS%'] = round(away_table['CS'] / away_table['played'],2)
    home_table['FG%'] = round(home_table['first_goal'] / home_table['played'],2)
    away_table['FG%'] = round(away_table['first_goal'] / away_table['played'],2)
    home_table['FC%'] = round(home_table['first_conc'] / home_table['played'],2)
    away_table['FC%'] = round(away_table['first_conc'] / away_table['played'],2)
    home_table = home_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    away_table = away_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    home_table = home_table.sort_values('PPG', ascending=False)
    away_table = away_table.sort_values('PPG', ascending=False)

    return home_table, away_table

eng_l10_table_h,eng_l10_table_a = home_away_table_form(eng_l10)
fra_l10_table_h,fra_l10_table_a = home_away_table_form(fra_l10)
ger_l10_table_h,ger_l10_table_a = home_away_table_form(ger_l10)
ita_l10_table_h,ita_l10_table_a = home_away_table_form(ita_l10)
spa_l10_table_h,spa_l10_table_a = home_away_table_form(spa_l10)

In [46]:
# Home/Away Tables rolling season (Historic)

def home_away_table(df):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1
    
    # new df
    df_h_a = df.groupby(["Team","H/A"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby("H/A")]
    away_table = h_a_sep[0].droplevel("H/A")
    home_table = h_a_sep[1].droplevel("H/A")
    home_table = home_table.apply(pd.to_numeric)
    away_table = away_table.apply(pd.to_numeric)
    home_table['GD'] = home_table['G'] - home_table['GC']
    away_table['GD'] = away_table['G'] - away_table['GC']
    home_table['PPG'] = round(home_table['Pts'] / home_table['played'],2)
    away_table['PPG'] = round(away_table['Pts'] / away_table['played'],2)
    home_table['BTTS%'] = round(home_table['BTTS'] / home_table['played'],2)
    away_table['BTTS%'] = round(away_table['BTTS'] / away_table['played'],2)
    home_table['Blank%'] = round(home_table['Blank'] / home_table['played'],2)
    away_table['Blank%'] = round(away_table['Blank'] / away_table['played'],2)
    home_table['CS%'] = round(home_table['CS'] / home_table['played'],2)
    away_table['CS%'] = round(away_table['CS'] / away_table['played'],2)
    home_table['FG%'] = round(home_table['first_goal'] / home_table['played'],2)
    away_table['FG%'] = round(away_table['first_goal'] / away_table['played'],2)
    home_table['FC%'] = round(home_table['first_conc'] / home_table['played'],2)
    away_table['FC%'] = round(away_table['first_conc'] / away_table['played'],2)

    home_table = home_table[~(home_table['played'] <= 10)]
    away_table = away_table[~(away_table['played'] <= 10)]
    home_table = home_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    away_table = away_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    home_table = home_table.sort_values('PPG', ascending=False)
    away_table = away_table.sort_values('PPG', ascending=False)

    return home_table, away_table

eng_table_h, eng_table_a = home_away_table(eng_df)
fra_table_h, fra_table_a = home_away_table(fra_df)
ger_table_h, ger_table_a = home_away_table(ger_df)
ita_table_h, ita_table_a = home_away_table(ita_df)
spa_table_h, spa_table_a = home_away_table(spa_df)

In [47]:
# Rank Finder (Strength of Opponent)

def find_ranks(home_df,away_df):
    q1_h = home_df['PPG'].quantile(0.25)
    q1_a = away_df['PPG'].quantile(0.25)
    q3_h = home_df['PPG'].quantile(0.75)
    q3_a = away_df['PPG'].quantile(0.75)
    home_df['Rank'] = home_df.apply(lambda x: 3 if x['PPG'] <= q1_h else( 1 if x['PPG'] >= q3_h else 2), axis=1) 
    away_df['Rank'] = away_df.apply(lambda x: 3 if x['PPG'] <= q1_a else( 1 if x['PPG'] >= q3_a else 2), axis=1) 

    return home_df,away_df

eng_rank_table_h,eng_rank_table_a = find_ranks(eng_table_h,eng_table_a)
fra_rank_table_h,fra_rank_table_a = find_ranks(fra_table_h,fra_table_a)
ger_rank_table_h,ger_rank_table_a = find_ranks(ger_table_h,ger_table_a)
ita_rank_table_h,ita_rank_table_a = find_ranks(ita_table_h,ita_table_a)
spa_rank_table_h,spa_rank_table_a = find_ranks(spa_table_h,spa_table_a)

In [48]:
# Apply rank to overall dataframe, and then group in tables by rank like in home vs away
def rank_to_df(rank_df_h,rank_df_a,df):
    rank_df_h['Team index'] = rank_df_h.index
    rank_df_a['Team index'] = rank_df_a.index

    rank1_h = rank_df_h.loc[rank_df_h['Rank'] == 1, ['Team index']].values.T.tolist()
    rank2_h = rank_df_h.loc[rank_df_h['Rank'] == 2, ['Team index']].values.T.tolist()
    rank3_h = rank_df_h.loc[rank_df_h['Rank'] == 3, ['Team index']].values.T.tolist()
    rank1_a = rank_df_a.loc[rank_df_a['Rank'] == 1, ['Team index']].values.T.tolist()
    rank2_a = rank_df_a.loc[rank_df_a['Rank'] == 2, ['Team index']].values.T.tolist()
    rank3_a = rank_df_a.loc[rank_df_a['Rank'] == 3, ['Team index']].values.T.tolist()

    df['Rank'] = df.apply(lambda x: 1 if x['H/A'] =="a" and x['Opp'] in rank1_h[0] else(1 if x['H/A'] == "h" and x['Opp'] in rank1_a[0] else
                                                                                     (2 if x['H/A'] == "a" and x['Opp'] in rank2_h[0] else
                                                                                      (2 if x['H/A'] == "h" and x['Opp'] in rank2_a[0] else
                                                                                       (3 if x['H/A'] == "a" and x['Opp'] in rank3_h[0] else 3
                                                                                       )))), axis=1) 
    return df

eng_df_updated = rank_to_df(eng_rank_table_h,eng_rank_table_a,eng_df)
fra_df_updated = rank_to_df(fra_rank_table_h,fra_rank_table_a,fra_df)
ger_df_updated = rank_to_df(ger_rank_table_h,ger_rank_table_a,ger_df)
ita_df_updated = rank_to_df(ita_rank_table_h,ita_rank_table_a,ita_df)
spa_df_updated = rank_to_df(spa_rank_table_h,spa_rank_table_a,spa_df)

In [49]:
def home_away_rank_table(df):
    # changes to df
    df['match_ID'] = df['ID'].apply(lambda x:x[-5:])
    df['match_ID'] = pd.to_numeric(df['match_ID'])
    df['played'] = 1
    
    # new df
    df_h_a = df.groupby(["Team","H/A","Rank"]).agg({'played':'sum', 'xG':'sum', 'xGC':'sum','G':'sum', 'GC':'sum', 'Pts':'sum',
                                             'S':'sum', 'SC':'sum', 'QS': 'sum','QSC':'sum',
                                             'BTTS':'sum', 'Blank':'sum', 'CS':'sum','first_goal':'sum','first_conc':'sum'})
    h_a_sep = [x for _, x in df_h_a.groupby(["H/A","Rank"])]
    r1_h_table = h_a_sep[3].droplevel("H/A")
    r2_h_table = h_a_sep[4].droplevel("H/A")
    r3_h_table = h_a_sep[5].droplevel("H/A")
    r1_a_table = h_a_sep[0].droplevel("H/A")
    r2_a_table = h_a_sep[1].droplevel("H/A")
    r3_a_table = h_a_sep[2].droplevel("H/A")

    r1_h_table = r1_h_table.apply(pd.to_numeric)
    r2_h_table = r2_h_table.apply(pd.to_numeric)
    r3_h_table = r3_h_table.apply(pd.to_numeric)
    r1_a_table = r1_a_table.apply(pd.to_numeric)
    r2_a_table = r2_a_table.apply(pd.to_numeric)
    r3_a_table = r3_a_table.apply(pd.to_numeric)
   
    r1_h_table['GD'] = r1_h_table['G'] - r1_h_table['GC']
    r2_h_table['GD'] = r2_h_table['G'] - r2_h_table['GC']
    r3_h_table['GD'] = r3_h_table['G'] - r3_h_table['GC']
    r1_a_table['GD'] = r1_a_table['G'] - r1_a_table['GC']
    r2_a_table['GD'] = r2_a_table['G'] - r2_a_table['GC']
    r3_a_table['GD'] = r3_a_table['G'] - r3_a_table['GC']

    r1_h_table['PPG'] = round(r1_h_table['Pts'] / r1_h_table['played'],2)
    r2_h_table['PPG'] = round(r2_h_table['Pts'] / r2_h_table['played'],2)
    r3_h_table['PPG'] = round(r3_h_table['Pts'] / r3_h_table['played'],2)
    r1_a_table['PPG'] = round(r1_a_table['Pts'] / r1_a_table['played'],2)
    r2_a_table['PPG'] = round(r2_a_table['Pts'] / r2_a_table['played'],2)
    r3_a_table['PPG'] = round(r3_a_table['Pts'] / r3_a_table['played'],2)
     
    r1_h_table['BTTS%'] = round(r1_h_table['BTTS'] / r1_h_table['played'],2)
    r2_h_table['BTTS%'] = round(r2_h_table['BTTS'] / r2_h_table['played'],2)
    r3_h_table['BTTS%'] = round(r3_h_table['BTTS'] / r3_h_table['played'],2)
    r1_a_table['BTTS%'] = round(r1_a_table['BTTS'] / r1_a_table['played'],2)
    r2_a_table['BTTS%'] = round(r2_a_table['BTTS'] / r2_a_table['played'],2)
    r3_a_table['BTTS%'] = round(r3_a_table['BTTS'] / r3_a_table['played'],2)
     
    r1_h_table['Blank%'] = round(r1_h_table['Blank'] / r1_h_table['played'],2)
    r2_h_table['Blank%'] = round(r2_h_table['Blank'] / r2_h_table['played'],2)
    r3_h_table['Blank%'] = round(r3_h_table['Blank'] / r3_h_table['played'],2)
    r1_a_table['Blank%'] = round(r1_a_table['Blank'] / r1_a_table['played'],2)
    r2_a_table['Blank%'] = round(r2_a_table['Blank'] / r2_a_table['played'],2)
    r3_a_table['Blank%'] = round(r3_a_table['Blank'] / r3_a_table['played'],2)
     
    r1_h_table['CS%'] = round(r1_h_table['CS'] / r1_h_table['played'],2)
    r2_h_table['CS%'] = round(r2_h_table['CS'] / r2_h_table['played'],2)
    r3_h_table['CS%'] = round(r3_h_table['CS'] / r3_h_table['played'],2)
    r1_a_table['CS%'] = round(r1_a_table['CS'] / r1_a_table['played'],2)
    r2_a_table['CS%'] = round(r2_a_table['CS'] / r2_a_table['played'],2)
    r3_a_table['CS%'] = round(r3_a_table['CS'] / r3_a_table['played'],2)
     
    r1_h_table['FG%'] = round(r1_h_table['first_goal'] / r1_h_table['played'],2)
    r2_h_table['FG%'] = round(r2_h_table['first_goal'] / r2_h_table['played'],2)
    r3_h_table['FG%'] = round(r3_h_table['first_goal'] / r3_h_table['played'],2)
    r1_a_table['FG%'] = round(r1_a_table['first_goal'] / r1_a_table['played'],2)
    r2_a_table['FG%'] = round(r2_a_table['first_goal'] / r2_a_table['played'],2)
    r3_a_table['FG%'] = round(r3_a_table['first_goal'] / r3_a_table['played'],2)

    r1_h_table['FC%'] = round(r1_h_table['first_conc'] / r1_h_table['played'],2)
    r2_h_table['FC%'] = round(r2_h_table['first_conc'] / r2_h_table['played'],2)
    r3_h_table['FC%'] = round(r3_h_table['first_conc'] / r3_h_table['played'],2)
    r1_a_table['FC%'] = round(r1_a_table['first_conc'] / r1_a_table['played'],2)
    r2_a_table['FC%'] = round(r2_a_table['first_conc'] / r2_a_table['played'],2)
    r3_a_table['FC%'] = round(r3_a_table['first_conc'] / r3_a_table['played'],2)

    r1_h_table = r1_h_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    r2_h_table = r2_h_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    r3_h_table = r3_h_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    r1_a_table = r1_a_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    r2_a_table = r2_a_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]
    r3_a_table = r3_a_table[['played','xG','xGC','G','GC','GD','Pts','PPG','S','SC','QS','QSC','BTTS%','Blank%','CS%','FG%','FC%']]

    r1_h_table = r1_h_table.sort_values('PPG', ascending=False)
    r2_h_table = r2_h_table.sort_values('PPG', ascending=False)
    r3_h_table = r3_h_table.sort_values('PPG', ascending=False)
    r1_a_table = r1_a_table.sort_values('PPG', ascending=False)
    r2_a_table = r2_a_table.sort_values('PPG', ascending=False)
    r3_a_table = r3_a_table.sort_values('PPG', ascending=False)

    return r1_h_table, r2_h_table, r3_h_table, r1_a_table, r2_a_table, r3_a_table

r1_h_eng,r2_h_eng,r3_h_eng,r1_a_eng,r2_a_eng,r3_a_eng = home_away_rank_table(eng_df_updated)
r1_h_fra,r2_h_fra,r3_h_fra,r1_a_fra,r2_a_fra,r3_a_fra = home_away_rank_table(fra_df_updated)
r1_h_ger,r2_h_ger,r3_h_ger,r1_a_ger,r2_a_ger,r3_a_ger = home_away_rank_table(ger_df_updated)
r1_h_ita,r2_h_ita,r3_h_ita,r1_a_ita,r2_a_ita,r3_a_ita = home_away_rank_table(ita_df_updated)
r1_h_spa,r2_h_spa,r3_h_spa,r1_a_spa,r2_a_spa,r3_a_spa = home_away_rank_table(spa_df_updated)

In [50]:
with pd.ExcelWriter("../EDA_DF/EPL_understat.xlsx") as writer:
    eng_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_eng.to_excel(writer, sheet_name="rank1_home")
    r2_h_eng.to_excel(writer, sheet_name="rank2_home")
    r3_h_eng.to_excel(writer, sheet_name="rank3_home")
    r1_a_eng.to_excel(writer, sheet_name="rank1_away")
    r2_a_eng.to_excel(writer, sheet_name="rank2_away")
    r3_a_eng.to_excel(writer, sheet_name="rank3_away")
    eng_l10_table_h.to_excel(writer, sheet_name="last10_home")
    eng_l10_table_a.to_excel(writer, sheet_name="last10_away")
    eng_table_h.to_excel(writer, sheet_name="home")
    eng_table_a.to_excel(writer, sheet_name="away")

with pd.ExcelWriter("../EDA_DF/Ligue1_understat.xlsx") as writer:
    fra_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_fra.to_excel(writer, sheet_name="rank1_home")
    r2_h_fra.to_excel(writer, sheet_name="rank2_home")
    r3_h_fra.to_excel(writer, sheet_name="rank3_home")
    r1_a_fra.to_excel(writer, sheet_name="rank1_away")
    r2_a_fra.to_excel(writer, sheet_name="rank2_away")
    r3_a_fra.to_excel(writer, sheet_name="rank3_away")
    fra_l10_table_h.to_excel(writer, sheet_name="last10_home")
    fra_l10_table_a.to_excel(writer, sheet_name="last10_away")
    fra_table_h.to_excel(writer, sheet_name="home")
    fra_table_a.to_excel(writer, sheet_name="away")

with pd.ExcelWriter("../EDA_DF/Bundesliga_understat.xlsx") as writer:
    ger_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_ger.to_excel(writer, sheet_name="rank1_home")
    r2_h_ger.to_excel(writer, sheet_name="rank2_home")
    r3_h_ger.to_excel(writer, sheet_name="rank3_home")
    r1_a_ger.to_excel(writer, sheet_name="rank1_away")
    r2_a_ger.to_excel(writer, sheet_name="rank2_away")
    r3_a_ger.to_excel(writer, sheet_name="rank3_away")
    ger_l10_table_h.to_excel(writer, sheet_name="last10_home")
    ger_l10_table_a.to_excel(writer, sheet_name="last10_away")
    ger_table_h.to_excel(writer, sheet_name="home")
    ger_table_a.to_excel(writer, sheet_name="away")

with pd.ExcelWriter("../EDA_DF/SerieA_understat.xlsx") as writer:
    ita_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_ita.to_excel(writer, sheet_name="rank1_home")
    r2_h_ita.to_excel(writer, sheet_name="rank2_home")
    r3_h_ita.to_excel(writer, sheet_name="rank3_home")
    r1_a_ita.to_excel(writer, sheet_name="rank1_away")
    r2_a_ita.to_excel(writer, sheet_name="rank2_away")
    r3_a_ita.to_excel(writer, sheet_name="rank3_away")
    ita_l10_table_h.to_excel(writer, sheet_name="last10_home")
    ita_l10_table_a.to_excel(writer, sheet_name="last10_away")
    ita_table_h.to_excel(writer, sheet_name="home")
    ita_table_a.to_excel(writer, sheet_name="away")
    
with pd.ExcelWriter("../EDA_DF/LaLiga_understat.xlsx") as writer:
    spa_df_updated.to_excel(writer, sheet_name="gamesheet")
    r1_h_spa.to_excel(writer, sheet_name="rank1_home")
    r2_h_spa.to_excel(writer, sheet_name="rank2_home")
    r3_h_spa.to_excel(writer, sheet_name="rank3_home")
    r1_a_spa.to_excel(writer, sheet_name="rank1_away")
    r2_a_spa.to_excel(writer, sheet_name="rank2_away")
    r3_a_spa.to_excel(writer, sheet_name="rank3_away")
    spa_l10_table_h.to_excel(writer, sheet_name="last10_home")
    spa_l10_table_a.to_excel(writer, sheet_name="last10_away")
    spa_table_h.to_excel(writer, sheet_name="home")
    spa_table_a.to_excel(writer, sheet_name="away")